# MeLi scraping

In [ ]:
#! pip install requests
#! pip install beautifulsoup4
#! pip install matplotlib
#! pip install ipykernel

In [ ]:
import utils_meli
from importlib import reload
import pandas as pd
import os
import re
from datetime import datetime
import plotly.express as px

from IPython.display import display
pd.options.display.max_colwidth = 170
pd.options.display.max_rows = None

In [ ]:
parent_path = r"".replace("\\", "/")

In [ ]:
reload(utils_meli)

In [ ]:
# 30 min aprox.
reload(utils_meli)
df = utils_meli.load_html_search()

In [ ]:
df.loc[4,'Comments']

In [ ]:
utils_meli.open_link(df,3)

In [ ]:
for idx in range(len(df)-1):
    for comm in df.loc[idx,'Comments']:
        if re.search(r'excelente', comm, flags=re.IGNORECASE):
            display(df.iloc[idx])

In [ ]:
utils_meli.open_link(df,250)

# Word Cloud

In [ ]:
import spacy
from unidecode import unidecode
from sklearn.feature_extraction.text import CountVectorizer
from PIL import Image
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
spacy.cli.download("es_core_news_sm")

In [ ]:
nlp = spacy.load("es_core_news_sm")
pat = re.compile(r"[^a-z ]")
spaces = re.compile(r"\s{2,}")

def preprocess(text, min_len=1, max_len=1000):
    # spacy Doc creation
    doc = nlp(text)
    # Remove stopwords
    filtered_tokens = filter(
            lambda token: not token.is_stop,
            doc
            )
    # Filter words by length and remove stop words
    filtered_tokens2 = filter(
            lambda token: len(token) >= min_len and len(token) <= max_len and not token.is_stop,
            filtered_tokens
        )
    # Lemmatization
    lemmas = map(
            lambda token: token.lemma_,
            filtered_tokens2
            )
    lemma_text = " ".join(lemmas)
    # Normalize text
    norm_text = unidecode(lemma_text)
    # Remove accents
    lower_text = norm_text.lower()
    # Remove special characters
    clean_text = re.sub(pat, "", lower_text)
    # Remomove duplicate spaces (if exist)
    spaces_text = re.sub(spaces, " ", clean_text)
    return spaces_text.strip()

In [ ]:
all_comments = []

for comm in df['Comments']:
    all_comments.extend(comm)

print(all_comments)

In [ ]:
prep_comments = list(map(preprocess,all_comments))
print(prep_comments)

In [ ]:
vect = (
    CountVectorizer(max_features=1000, max_df=0.7)
    .fit(prep_comments)
    )

X = vect.transform(prep_comments)
vocab = vect.get_feature_names_out()

counts = np.array(X.sum(axis=0)).flatten()
counts_dict = {word: count for word, count in zip(vocab, counts)}

mask = np.array(Image.open(r"\cloud.png"))

In [ ]:
wc = WordCloud(background_color='white',
        width=3000,
        height=2000,
        collocations=False,
        mask=mask,
        colormap = 'Dark2',
        max_words=40
).generate_from_frequencies(counts_dict)

plt.figure(figsize=[7,7])
plt.imshow(wc,interpolation="bilinear")
plt.axis("off") 
plt.show()

In [ ]:
utils_meli.open_link(df,4)

In [ ]:
df.to_excel(os.path.join(parent_path,"_{}_{}_{}.xlsx".format(datetime.today().year,datetime.today().month,datetime.today().day)), index=False)

In [ ]:
# Pass the DataFrame and copy and paste the product name you want to see in browser
reload(utils_meli)
utils_meli.open_link(df,'')

In [ ]:
# Pass the DataFrame and index you want to open in browser
reload(utils_meli)
utils_meli.open_link(df,3)

# Metrics

## Price Analysis

In [ ]:
df = pd.read_excel(r"".replace("\\", "/"))
df.head()

In [ ]:
reload(utils_meli)
df_25 = utils_meli.quartile_prices(df)[0]
df_25_50 = utils_meli.quartile_prices(df)[1]
df_50_75 = utils_meli.quartile_prices(df)[2]
df_75 = utils_meli.quartile_prices(df)[3]
quartiles = utils_meli.quartile_prices(df)[4]
print(f'Median: {df.Price.median()}')
quartiles

In [ ]:
# Products above quartile 75
print('Prime products [above quartile 75] -> {} products ({}%) '.format(df_75.Product.to_list().__len__(), round((df_75.Product.to_list().__len__() / df.__len__()) * 100, 2)))
print('Premium products [between quartiles 50 & 75] -> {} products ({}%) '.format(df_50_75.Product.to_list().__len__(), round((df_50_75.Product.to_list().__len__() / df.__len__()) * 100, 2)))
print('Mid products [between quartiles 25 & 50] -> {} products ({}%) '.format(df_25_50.Product.to_list().__len__(), round((df_25_50.Product.to_list().__len__() / df.__len__()) * 100, 2)))
print('Low products [below quartile 25 ] -> {} products ({}%) '.format(df_25.Product.to_list().__len__(), round((df_25.Product.to_list().__len__() / df.__len__()) * 100, 2)))

In [ ]:
print('Quartile (75>) - Products: {}'.format(df_75.__len__()))
display(df_75.head())
display(px.bar(df_75.Brand.value_counts().to_frame()[df_75.Brand.value_counts().to_frame()['Brand']>=10], x='Brand', width=1100, height=600, text_auto=True,
       labels={'index':'Brand', 'Brand':'Count'}).update_layout(xaxis={'categoryorder':'total descending'}, title='Brands over 10 search results'))

print('Quartile (50-75] - Products: {}'.format(df_50_75.__len__()))
display(df_50_75.head())
display(px.bar(df_50_75.Brand.value_counts().to_frame()[df_50_75.Brand.value_counts().to_frame()['Brand']>=10], x='Brand', width=1100, height=600, text_auto=True,
       labels={'index':'Brand', 'Brand':'Count'}).update_layout(xaxis={'categoryorder':'total descending'}, title='Brands over 10 search results'))

print('Quartile (25-50] - Products: {}'.format(df_25_50.__len__()))
display(df_25_50.head())
display(px.bar(df_25_50.Brand.value_counts().to_frame()[df_25_50.Brand.value_counts().to_frame()['Brand']>=10], x='Brand', width=1100, height=600, text_auto=True,
       labels={'index':'Brand', 'Brand':'Count'}).update_layout(xaxis={'categoryorder':'total descending'}, title='Brands over 10 search results'))

print('Quartile (<25] - Products: {}'.format(df_25.__len__()))
display(df_25.head())
display(px.bar(df_25.Brand.value_counts().to_frame()[df_25.Brand.value_counts().to_frame()['Brand']>=10], x='Brand', width=1100, height=600, text_auto=True,
       labels={'index':'Brand', 'Brand':'Count'}).update_layout(xaxis={'categoryorder':'total descending'}, title='Brands over 10 search results'))

In [ ]:
px.box(df,'Price', title=f'Price distribution')

In [ ]:
print(df[df['Price'] <= 831].shape[0])
print(df[df['Price'] >= 449].shape[0] / df.shape[0])
print(df[(df['Price'] > 247) & (df['Price'] <= 449)].shape[0] / df.shape[0])

In [ ]:
display(px.histogram(df, x='Price', title=f'Price distribution'))
display(px.histogram(df_75, x='Price', title=f'Price distribution - Prime Products', nbins=10, text_auto=True))
display(df_75.describe().round(2))
display(px.histogram(df_50_75, x='Price', title=f'Price distribution - Premium Products', nbins=15, text_auto=True))
display(df_50_75.describe().round(2))
display(px.histogram(df_25_50, x='Price', title=f'Price distribution - Mid Products', nbins=15, text_auto=True))
display(df_25_50.describe().round(2))

In [ ]:
#Prime brands
display(df_75.Brand.value_counts().to_frame().reset_index().rename(columns={'index':'Brand','Brand':'Results'}).head(10))
'''for i in df_75.Brand.value_counts().to_frame().reset_index().rename(columns={'index':'Brand','Brand':'Results'}).head(10).Brand.tolist():
    print(i)'''

#Premium brands
display(df_50_75.Brand.value_counts().to_frame().reset_index().rename(columns={'index':'Brand','Brand':'Results'}).head(10))
'''for i in df_50_75.Brand.value_counts().to_frame().reset_index().rename(columns={'index':'Brand','Brand':'Results'}).head(10).Brand.tolist():
    print(i)'''

#Mid brands
display(df_25_50.Brand.value_counts().to_frame().reset_index().rename(columns={'index':'Brand','Brand':'Results'}).head(10))
'''for i in df_25_50.Brand.value_counts().to_frame().reset_index().rename(columns={'index':'Brand','Brand':'Results'}).head(10).Brand.tolist():
    print(i)'''

In [ ]:
a = df_25_50.Brand.value_counts().to_frame().reset_index().rename(columns={'index':'Brand','Brand':'Results'}).head(10)
a['Proc'] = round((a.Results / a.Results.sum())*100,2)
a

## Brands distribution

In [ ]:
df.columns

In [ ]:
px.bar(df.Brand.value_counts().to_frame()[df.Brand.value_counts().to_frame()['Brand']>=10], x='Brand', width=1100, height=600, text_auto=True,
       labels={'index':'Brand', 'Brand':'Count'}).update_layout(xaxis={'categoryorder':'total descending'}, title='Brands over 10 search results')

In [ ]:
print(df[df.Brand!='-'].Brand.unique().tolist())

## Brand appearences

In [ ]:
# Look for specif word(s)
brand_products = []

for i in df[df['Score']==5]['Product']:
    if (re.search(r'', i, flags=re.IGNORECASE)) or (re.search(r'', i, flags=re.IGNORECASE)) or (re.search(r'', i, flags=re.IGNORECASE)) or (re.search(r'', i, flags=re.IGNORECASE)) \
       or (re.search(r'', i, flags=re.IGNORECASE)) or (re.search(r'', i, flags=re.IGNORECASE)) or (re.search(r'', i, flags=re.IGNORECASE)) or (re.search(r'', i, flags=re.IGNORECASE)):
        brand_products.append(i)
        
df_filtered = df[df.Product.isin(brand_products)].reset_index(drop=True)
df_filtered = df_filtered.drop_duplicates(subset=['Product', 'Price', 'Score', 'Ratings'], keep='first',ignore_index=True)

print(df_filtered.shape)
df_filtered

In [ ]:
df_filtered.to_excel(os.path.join(parent_path,"_{}_{}_{}.xlsx".format(datetime.today().year,datetime.today().month,datetime.today().day)), index=False)

In [ ]:
count = 0
brand_producs = []
brand_prices = []
for i in range(df.__len__()):
    if (re.search('', df.loc[i,'Product'], flags=re.IGNORECASE)) or (re.search('', df.loc[i,'Product'], flags=re.IGNORECASE)) or (re.search('', df.loc[i,'Product'], flags=re.IGNORECASE)):
        count += 1
        brand_producs.append(df.loc[i,'Product'])
        brand_prices.append(df.loc[i,'Price'])
print(count)

brand_df = pd.DataFrame(data={'Product':brand_producs, 'Price':brand_prices})
print(round(brand_df.Price.mean(), 2))
brand_df.head(10)

----------------------------------------

In [ ]:
import pandas as pd
import webbrowser
import requests
import bs4
from IPython.display import display

In [ ]:
search = input('Insert your search:')
url = 'https://listado.mercadolibre.com.mx/{}#D[A:{}]'.format(search.replace(' ','-'), search)
request = requests.get(url)
print(f'Response status code: {request.status_code}')
content = request.text
soup = bs4.BeautifulSoup(content, 'html.parser') # HTML document

total_pages = int(soup.find_all("div", {"class":"ui-search-pagination shops__pagination-content"})[0].find(class_="andes-pagination__page-count").text.replace("de ",""))
current_page = soup.find_all("div", {"class":"ui-search-pagination shops__pagination-content"})[0].find("li").text # See actual page
print(f'***************{soup.title.text}***************')
print('Total pages: {}'.format(total_pages))
print('Current page: {}\n\n'.format(current_page))

brands = []
products = []
prices = []
scores = []
comments = []
links = []

for page in range(1,total_pages+1):

    if page == 1:
        url_page = 'https://listado.mercadolibre.com.mx/{}_NoIndex_True'.format(search.replace(' ','-'))
        request_page = requests.get(url_page)
        content_page = request_page.text
        soup_page = bs4.BeautifulSoup(content_page, 'html.parser')
        tag_filter_page = soup_page.find_all("div", {"class":"ui-search-result__content-wrapper shops__result-content-wrapper"})

        for tag in tag_filter_page:
            brands.append(tag.find(class_="ui-search-item__brand-discoverability ui-search-item__group__element shops__items-group-details").text)
            products.append(tag.find("h2").text)
            prices.append(tag.find(class_="price-tag-fraction").text)
            links.append(tag.find("a")["href"])

    else:
        url_page = 'https://listado.mercadolibre.com.mx/{}_Desde_{}_NoIndex_True'.format(search.replace(' ','-'),(page*50)+1)
        request_page = requests.get(url_page)
        content_page = request_page.text
        soup_page = bs4.BeautifulSoup(content_page, 'html.parser')
        tag_filter_page = soup_page.find_all("div", {"class":"ui-search-result__content-wrapper shops__result-content-wrapper"})

        for tag in tag_filter_page:
            brands.append(tag.find(class_="ui-search-item__brand-discoverability ui-search-item__group__element shops__items-group-details").text)
            products.append(tag.find("h2").text)
            prices.append(tag.find(class_="price-tag-fraction").text)
            links.append(tag.find("a")["href"])
        

data = {}
data["Brand"] = brands
data['Product'] = products
data['Price'] = prices
'''data['Score'] = scores
data['Comentarios'] = comments'''
data["Link"] = links

df = pd.DataFrame(data = data)
df = df.drop_duplicates(ignore_index=True)
df['Brand'] = df['Brand'].apply(lambda x: "-" if x == '' else x)
df['Price'] = df['Price'].apply(lambda x: x.replace(",",""))
df['Price'] = df['Price'].astype(int)
print('Total products retrieved: {}'.format(df.shape[0]))
display(df.head())

In [ ]:
request_ = requests.get('https://listado.mercadolibre.com.mx/iphone-3#D[A:iphone%203]')
print(f'Response status code: {request_.status_code}')
content_ = request_.text
soup_ = bs4.BeautifulSoup(content_, 'html.parser') # HTML document
total_pages = soup_.find_all("div", {"class":"ui-search-pagination shops__pagination-content"})#[0].find(class_="andes-pagination__page-count").text.replace("de ",""))
print('Total pages: {}'.format(total_pages))
print(soup_.title.text)

In [ ]:
soup_.find_all("div", {"class":"ui-search-pagination shops__pagination-content"}).__len__()

In [ ]:
soup_.find_all('article', {"class":"ui-review-capability-comments__comment"})

In [ ]:
v = []
l = soup_.find_all('article', {"class":"ui-review-capability-comments__comment"}) #[2].find('p', class_="ui-review-capability-comments__comment__content").text
print(l.__len__())
for c in l:
    v.append(c.find('p', class_="ui-review-capability-comments__comment__content").text)
print(l.__len__())
v

In [ ]:
search = input('Insert your search:')
print(f'Keywords: {search.replace("-"," ")}')
url = 'https://listado.mercadolibre.com.mx/{}#D[A:{}]'.format(search.replace(' ','-'), search)
request = requests.get(url)
content = request.text
soup = bs4.BeautifulSoup(content, 'html.parser') # HTML document

brands = []
products = []
prices = []
links = []

pages_flag = soup.find_all("div", {"class":"ui-search-pagination shops__pagination-content"})


if pages_flag.__len__() != 0:
    print(f'Response status code: {request.status_code}')
    total_pages = int(soup.find("div", class_="ui-search-pagination shops__pagination-content").find("li",class_="andes-pagination__page-count").text.replace("de ",""))
    print(f'***************{soup.title.text}***************')
    print('Total pages: {}'.format(total_pages))

    for page in range(1,total_pages+1):
        if page == 1:
            url_page = 'https://listado.mercadolibre.com.mx/{}_NoIndex_True'.format(search.replace(' ','-'))
            request_page = requests.get(url_page)
            content_page = request_page.text
            soup_page = bs4.BeautifulSoup(content_page, 'html.parser')
            tag_filter_page = soup_page.find_all("div", {"class":"ui-search-result__content-wrapper shops__result-content-wrapper"})

            while soup_page.find_all("div", {"class":"ui-search-result__content-wrapper shops__result-content-wrapper"}).__len__() == 0:
                url_page = 'https://listado.mercadolibre.com.mx/{}_NoIndex_True'.format(search.replace(' ','-'))
                request_page = requests.get(url_page)
                content_page = request_page.text
                soup_page = bs4.BeautifulSoup(content_page, 'html.parser')
                tag_filter_page = soup_page.find_all("div", {"class":"ui-search-result__content-wrapper shops__result-content-wrapper"})

            # for tag in tag_filter_page:
            #     brands.append(tag.find(class_="ui-search-item__brand-discoverability ui-search-item__group__element shops__items-group-details").text)
            #     products.append(tag.find("h2").text)
            #     prices.append(tag.find(class_="price-tag-fraction").text)
            #     links.append(tag.find("a")["href"])

        else:
            url_page = 'https://listado.mercadolibre.com.mx/{}_Desde_{}_NoIndex_True'.format(search.replace(' ','-'),(page*50)+1)
            request_page = requests.get(url_page)
            content_page = request_page.text
            soup_page = bs4.BeautifulSoup(content_page, 'html.parser')
            tag_filter_page = soup_page.find_all("div", {"class":"ui-search-result__content-wrapper shops__result-content-wrapper"})

            while soup_page.find_all("div", {"class":"ui-search-result__content-wrapper shops__result-content-wrapper"}).__len__() == 0:
                url_page = 'https://listado.mercadolibre.com.mx/{}_Desde_{}_NoIndex_True'.format(search.replace(' ','-'),(page*50)+1)
                request_page = requests.get(url_page)
                content_page = request_page.text
                soup_page = bs4.BeautifulSoup(content_page, 'html.parser')
                tag_filter_page = soup_page.find_all("div", {"class":"ui-search-result__content-wrapper shops__result-content-wrapper"}) 

            # for tag in tag_filter_page:
            #     products.append(tag.find("h2").text)
            #     prices.append(tag.find(class_="price-tag-fraction").text)
            #     links.append(tag.find("a")["href"])
            #     brands.append(tag.find(class_="ui-search-item__brand-discoverability ui-search-item__group__element shops__items-group-details").text)
    
        # current_page = soup.find_all("div", {"class":"ui-search-pagination shops__pagination-content"})[0].find("li").text # See actual page
        print('Scraping page {}/{}'.format(page,total_pages))

        for tag in tag_filter_page:
            brands.append(tag.find(class_="ui-search-item__brand-discoverability ui-search-item__group__element shops__items-group-details").text)
            products.append(tag.find("h2").text)
            prices.append(tag.find("span",class_="andes-money-amount__fraction").text)
            # prices.append(tag.find(class_="price-tag-fraction").text)
            links.append(tag.find("a")["href"])

    data = {}
    data["Brand"] = brands
    data['Product'] = products
    data['Price'] = prices
    data["Link"] = links

    df = pd.DataFrame(data = data)
    df = df.drop_duplicates(ignore_index=True)
    df['Brand'] = df['Brand'].apply(lambda x: "-" if x == '' else x)
    df['Price'] = df['Price'].apply(lambda x: x.replace(",",""))
    df['Price'] = df['Price'].astype(int)

    display(df.head(3))